# The fast Fourier transform

We consider a question: ***How to Efficiently compute multiplication of polunomial***?
i.e.
> For two given degree-$d$ polynomial $A(x)=\sum^d_{i=0}a_ix^i, B(x)=\sum^d_{i=0}b_ix^i$, we want to compute the product of them:
> $$
C(x)=A(x)\cdot B(x)=\sum^{2d}_{i=0}c_ix^i\\
where\ c_i=\sum_{j=0}^ia_jb_{i-j}
> $$

Obviously there is a fact that:
> A degree-$d$ polynomial $A(x)=\sum^d_{i=0}a_ix^i$ is determinated by $d+1$ values:
> 1. Its coefficients:
> $$
a_0,a_1,\dots a_d
> $$
> 2. Its distinct pairs(key,value)
> $$
> (x_0, A(x_0)),\dots, (x_{d+1},A(x_{d+1}))
> $$

If we calculate the coefficients of $C(x)$.  
i.e. by the equation $c_i=\sum_{j=0}^ia_jb_{i-j}$, then the time complexity is $O(m^2)$. 


In [8]:
class Polynomial:
    def __init__(self, coefficients: list) -> None:
        coefficients = coefficients.copy()
        while len(coefficients) > 1 and coefficients[-1] == 0:
            coefficients.pop()
        self.coefficients = coefficients# From the constant item to the heightest degree
        self.degree = len(coefficients) - 1
        
    def __call__(self, x):
        return self.value_of(x)
    
    def value_of(self, x):
        result = 0
        for i, c in enumerate(self.coefficients):
            result += c * (x ** i)
        return result
    
    def __bool__(self)-> bool:
        return any(c != 0 for c in self.coefficients)
    
    def __eq__(self, value: object) -> bool:
        if not isinstance(value, Polynomial):
            return False
        return self.coefficients == value.coefficients
    
    def __str__(self) -> str:
        result = ""
        for i,c in enumerate(self.coefficients):
            if i == 0: 
                result = str(c)
            elif i == 1:
                result += f" + {c}x"
            else:
               result += f" + {c}x^{i}"
        return result
    
    def __repr__(self) -> str:
        return f"Polynomial({self.coefficients})"
    
    def __add__(self, other: object)-> 'Polynomial':
        if isinstance(other, (int, float)):
            newCoeffs = self.coefficients.copy()
            newCoeffs[0] += other
            return Polynomial(newCoeffs)
        elif isinstance(other, Polynomial):
            newDegree = max(self.degree, other.degree)
            newCoeffs = []
            for i in range(newDegree + 1):
                tem = 0.0
                if i < len(self.coefficients):
                    tem += self.coefficients[i]
                if i < len(other.coefficients):
                    tem += other.coefficients[i]
                newCoeffs.append(tem)
            return Polynomial(newCoeffs)
        else:
            return NotImplemented
        
    def __radd__(self, other: object)-> 'Polynomial':
        return self + other
    
    def __sub__(self, other: object)-> 'Polynomial':
        if isinstance(other, (int, float)):
            newCoeffs = self.coefficients.copy()
            newCoeffs[0] -= other
            return Polynomial(newCoeffs)
        elif isinstance(other, Polynomial):
            newDegree = max(self.degree, other.degree)
            newCoeffs = []
            for i in range(newDegree + 1):
                tem = 0.0
                if i < len(self.coefficients):
                    tem += self.coefficients[i]
                if i < len(other.coefficients):
                    tem -= other.coefficients[i]
                newCoeffs.append(tem)
            return Polynomial(newCoeffs)
        else:
            return NotImplemented
        
    def __neg__(self) -> 'Polynomial':
        newCoeff = [-self.coefficients[i] for i in range(self.degree + 1)]
        return Polynomial(newCoeff)
        
    def __rsub__(self, other: object)-> 'Polynomial':
        return - (self - other)
        
    
    def __mul__(self, other: object)-> 'Polynomial':
        if isinstance(other, (int, float)):
            newCoeffs = [self.coefficients[i] * other for i in range(self.degree + 1)]
            return Polynomial(newCoeffs)
        elif isinstance(other, Polynomial):
            newDegree = self.degree + other.degree
            newCoeffs = []
            for i in range(newDegree + 1):
                tem = 0.0
                for j in range(i + 1):
                    if (j > self.degree) or (i-j > other.degree):
                        continue
                    tem += self.coefficients[j] * other.coefficients[i-j]
                newCoeffs.append(tem)
            return Polynomial(newCoeffs)
        else:
            return NotImplemented
        
    def __rmul__(self, other: object)-> 'Polynomial':
        return self * other
    
    def __pow__(self, power: int)-> 'Polynomial':
        if not isinstance(power, int) or power < 0:
            return NotImplemented
        if power == 0:
            return Polynomial([1])
        result = Polynomial([1])
        base = self
        while power > 0:
            if power % 2 == 1:
                result = result * base
            base = base * base
            power //= 2
        return result

But what if we condiser the values of $A(x)$ and $B(x)$?
## Compute the products of values!

Its seems more simple. We need 4 steps to product two polynomial:
+ Polunomial Multiplication
  + Input: Coefficients of two polynomials, $A(x)$ and $B(x)$, of degree $d$
  + Output: Their product $C=A·B$
  + **Selection**
    + Pick some points $x_0,x_1,...,x_{n−1}$, where $n≥2d+1$
  + **Evaluation**
    + Compute $A(x_0),A(x_1),...,A(x_{n−1})$ and $B(x_0),B(x_1),...,B(x_{n−1})$
  + **Multiplication**
    + Compute $C(x_k) = A(x_k)B(x_k)$ for all $k=0,...,n−1$
  + **Interpolation**
    + Recover $C(x) = c_0 + c_1x+···+ c_2^dx_2^d$

We skip the selection, analyse the evaluation first.
## 1. Evaluation
When we directly compute the value of each point of $A(x)=\sum^d_{i=0}a_ix^i$, it's a $O(d)$ scale problem, that we need compute all  $d$ items of the polynomial. But consider the following case:
> If we choose two point $t$ and $-t$, then we have
> $$
\begin{aligned}
&A(t) = \sum^d_{i=0}a_it^i\\
&A(-t) = \sum^d_{i=0}a_i(-t)^i = \sum^d_{\text{i is even}}a_it^i - \sum^d_{\text{i is odd}}a_it^i\\
\end{aligned}
> $$ 

More generally, we define that:
$$
A(x) = A_{e}(x) + xA_o(x^2)
$$
where $A_e$ donates the even-numbered coeffcients(e.g. $a_0, a_2,\dots$), $A_o$ donates the odd-numbered coeffcients.